<a href="https://colab.research.google.com/github/yyc0314/Fooood-map/blob/main/sentiment%20analysis/ChatGPT_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import csv

r_100_path="/content/drive/Othercomputers/我的筆記型電腦/NLP/dynamic_crawler/rs_100.csv"
r_100=pd.read_csv(r_100_path)

r_comment=list(r_100["comment"])

#讀取rs_100.csv的欄
name=list(r_100["restaurant_name"])
href=list(r_100["href"])
tag=list(r_100["tag"])
star=list(r_100["star"])

In [ ]:
restaurants={}

In [ ]:
i=0

dic_cmt={}
for rs_name in name:
    if (r_comment[i])=='[]':
        print(i,r_comment[i],name[i])
    else:
        #print(r_comment[i])
        restaurants[rs_name]={} #新建立一個以餐廳名為key的dic

        restaurants[rs_name]["href"]=href[i]

        restaurants[rs_name]["tag"]=tag[i]
        restaurants[rs_name]["star"]=star[i]

        restaurants[rs_name]["emo"]=[]
        restaurants[rs_name]["score"]=[]
        restaurants[rs_name]["good"]=[]
        restaurants[rs_name]["bad"]=[]

        restaurants[rs_name]["if_gtp"]=0

        dic_cmt[rs_name]=[]

    i+=1

4 [] 品三國草船借箭
9 [] Ripple 義法西餐廳


In [ ]:
#存切過的cmt

i=0
cnt=0
for rs_name in dic_cmt:
    cm_split=[]
    if r_comment[i]=="[]":
        print(i)
        i+=1
    all_cmt=r_comment[i]

    cm_split=all_cmt.split(",")

    #print(cm_split)

    for cm in cm_split:
        if(len(cm) >= 30) and (len(cm) <= 80):
            dic_cmt[rs_name].append(cm)
    i+=1
    if i >= len(dic_cmt):
        break


4
9


In [ ]:
#算幾則評論、長度為多少
cnt=0
num=0
for i in dic_cmt:
    #print(i)
    for j in dic_cmt[i]:
        cnt+=len(j)
        #print(j)
        num+=1
print(cnt)
print(num)

656798
13599


In [ ]:
import openai
import sys

openai.api_key = ''　##

In [ ]:
#def_model="Plutchik情緒模型中情緒分為24種"#，分別是[狂喜/快樂/恬靜,警戒/期待/興趣,狂怒/憤怒/困惱,反感/厭惡/無聊,喪慟/悲傷/苦惱,驚異/驚訝/分心,恐怖/恐懼/擔心,渴慕/信任/認命]"
message='提供評論，以Plutchik情緒模型24種情緒詞彙分析["最符合此文本的情緒1","第二符合情緒2"] 極度的喜悅、滿足等正向辭彙為10分，極度不滿、憤怒等負面詞彙為0分，將情緒轉成一位小數 輸出: 情緒1,情緒2|情緒1轉換數值,情緒2轉換數值|最滿意的地方(10字內，可寫無),最不滿意的地方(10字內，可寫無)| 。 舉例: 狂喜,信任|9.2,8.5|服務佳,無| 或 興趣,擔心|5.4,3.5|麵好吃,衛生須注意| 評論:'

comment = "外帶肥腸酸辣米線，米線即使外帶回家也仍舊Q彈有嚼勁不輸拉麵，湯頭部份店家正常口味為中辣，個人不擅吃辣選小辣，光小辣就覺得辣到不行，是每次幾口就會被辣油嗆到咳嗽的程度，辣油很厚一層，辣味最重酸味其次麻感微微，有吃到花椒粒。肥腸沒有異味。生意好但出餐速度也很快"
comment2 ="港點是個好選擇，看到評論不錯就想說來試試看，老闆是個很爽朗的大姐，餐點普遍都還不錯，只是那個燴飯吃起來像一般的沙茶燴飯感覺，表現普通，也有提供免費紅茶算是很佛心，偶爾想吃個小點可以來坐坐"
comment3 ="🦆乾鍋鴨頭味美口感佳|一整個嘎蹦脆|🥤內用一壺酸梅湯|不會太甜|打卡免費贈送一壺|單點一壺80元|熱炒部分表現也好搶眼|「三杯雞」口感軟嫩|「口水雞」十分下飯|「酸辣土豆絲」感覺很獨特|是脆的|但後勁微微偏辣|整場吃下來最驚訝的是乾鍋鴨頭起鍋後加入的龍鳳高湯|融合在一起湯頭十分濃郁|有一點像牛奶豚骨湯|又帶著微微麻香味卻不辣|店內用餐也有包廂可以接待6人以上的團體聚餐|店內週邊附設兩個停車場|非常便利|想用餐的人記得查一下大宅門停車場|不要在週邊花冤枉錢去停車囉|因為附近停車格也不好找|其他停車場收費非常不友善|請留意您的荷包|"
comment4 ="好多年前來過一次|這次覺得口味有調整|比之前好吃|生意很不錯|二樓也幾乎都坐滿了|竟然還有素食套餐|很加分|上菜超快|小小問題是桌子不大|上個三四道菜就很擠了|較大一點的圓桌只有一個|其他都是方桌2022.02.18-晚餐 "

In [ ]:
#定義呼叫chatgpt的函式
def chatgpt(comment):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": message + comment}
        ],#message+"\n"+comment,
        max_tokens=100,
        temperature=0.7,
    )
    return (completion["choices"][0]["message"]["content"])

In [ ]:
chatgpt(comment2)

'喜悅,信任|9.0,8.0|餐點佳,燴飯普通|'

In [ ]:
"""#測試
completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": message + "\n" + comment3}
        ],#message+"\n"+comment,
        max_tokens=100,
        temperature=0.6,
    )
print(completion["choices"][0]["message"]["content"])
print(completion)
print(len(completion["choices"][0]["message"]["content"]))
print(len(message))"""

'#測試\ncompletion = openai.ChatCompletion.create(\n        model="gpt-3.5-turbo",\n        messages=[\n            {"role": "user", "content": message + "\n" + comment3} \n        ],#message+"\n"+comment,\n        max_tokens=100,\n        temperature=0.6,\n    )\nprint(completion["choices"][0]["message"]["content"])\nprint(completion)\nprint(len(completion["choices"][0]["message"]["content"]))\nprint(len(message))'

In [ ]:
dic_emo={}
for r_name in dic_cmt:
    dic_emo[r_name]=[]
print(len(dic_emo))

98


In [ ]:
start=0

In [ ]:
for r_name in dic_cmt:
    print(r_name)

大宅門乾鍋鴨頭
馬蹄鐵餐飲集團裕誠店
Legends Bar & Grill
冒煙的喬 高雄自由店
諾比饗美式小館
銅花精緻涮涮鍋
【簡單就好】高雄左營川菜餐廳|左營餐廳|左營熱炒|左營海鮮餐廳(館)推薦，左營平價捷運站美食地圖for晚餐消夜.restaurant and dinner
城市部落原住民餐廳－高雄店
美味素食~滬裕美食小館.愛地球餐廳
集品醬義大利麵/燉飯/焗烤/美食餐廳
悅品中餐廳/博愛館
薄多義 高雄裕誠店
高雄瑪莉桃桃親子生活餐廳|特色餐廳|親子餐廳|三民便當|包場餐廳|聚會餐廳|熱門美食
Pizza Rock Fumin Kaohsiung 高雄富民店
B D W 寶德旺 Tea&Coffee
高廬法式專廚餐廳
Ba guo巴鍋-個人鍋物/寵物友善/家庭聚餐/團體包場/小火鍋/英式風格餐廳/平價小火鍋
東方宴餐廳
紅葉海產（鼎山店）
芳師傅私房菜
夏佐廚房咖啡廳
泰灣泰式料理
麥當勞-高雄博愛二餐廳
東坡醉月餐館
泰泰餐桌 Thai Table
麥當勞-高雄民族餐廳
MASIDA韓食堂(瑪嘻大）
勝澤家（高雄鼎中店）丼飯、咖哩專賣店
丸亀製麵 高雄大樂店 烏龍麵餐廳
南瓜歐風咖哩 河堤店
熱浪島南洋蔬食茶堂（高雄店）
肯德基KFC-高雄崇德餐廳
義食光
麥當勞-鼎山家樂福餐廳
鰻魚日記 MIAMANDAY河堤店
鴨點棧烤鴨港點專賣
心滿義廚
🍗Crisken脆司肯美式炸雞
ChuJu waffle 雛菊鬆餅
饗泰多 Siam More 泰式風格餐廳 高雄義享店
時蒔 vegan 蔬食 麵館
財團法人喜憨兒社會福利基金會喜憨兒童園藝餐廳
吉野家 高雄大樂店
Sunny Queen陽光皇后義享店(義式料理、早午餐、甜點餐廳)
翠王香港茶餐廳
無毒世界蔬食餐廳（原無毒的家自由店國民旅遊卡特約店）
旭集 和食集錦 義享店
小團圓功夫私廚
有夠義式義大利麵餐館
古家海鮮
墨吉日本料理
THE LIN(1)
品粵私房菜
Skylark洋食‧芳鄰 義享店
京悅軒中餐廳
藍記小館
眺吧餐酒館 The Lookout Bistro & Bar
藝居酒屋
(高雄外燴專家)大都邑餐廳
Belon貝隆餐廳
帕狄尼諾 Padrino 義大利廚房
17西班牙餐酒館
舒醍雅鄰家廚房概念餐廳
暖呼呼食堂
Lion義式餐酒館
王品牛排高雄中正店
4分之一percent_rest

In [ ]:
import time
i=0

for r_name in dic_cmt: #餐廳名

    if restaurants[r_name]["if_gtp"]==0:
        restaurants[r_name]["if_gtp"]=1

        num=0

        for cmt in dic_cmt[r_name]: #cmt->str,in 評論list
            err_cnt=0
            while True:
                try:
                    text=chatgpt(cmt)
                    break
                except openai.error.APIError as e:
                    #Handle API error here, e.g. retry or log
                    print(f"OpenAI API returned an API Error: {e}")
                    time.sleep(20)
                    err_cnt+=1
                    pass
                except openai.error.APIConnectionError as e:
                    #Handle connection error here
                    print(f"Failed to connect to OpenAI API: {e}")
                    time.sleep(20)
                    err_cnt+=1
                    pass
                except openai.error.RateLimitError as e:
                    #Handle rate limit error (we recommend using exponential backoff)
                    print(f"OpenAI API request exceeded rate limit: {e}")
                    time.sleep(20)
                    err_cnt+=1
                    pass
                if err_cnt>3:
                    break

            if err_cnt<=3:
                dic_emo[r_name].append(text)

                response=text.split("|")
                print(cmt,num,"no.",i)
                #print(len(response))

                for w in range(0,3):
                    if len(response)>4 or w>=len(response):
                        break
                    one=response[w].split(",")
                    if w==0:
                        for emo in one:
                            restaurants[r_name]["emo"].append(emo)
                            print(emo,end=",")

                    elif w==1:
                        for score in one:
                            restaurants[r_name]["score"].append(score)
                            print(score,end=",")

                    elif w==2:
                        if len(one)<2:
                            break
                        restaurants[r_name]["good"].append(one[0])
                        restaurants[r_name]["bad"].append(one[1])
                        print(one[0],",",one[1])

            num+=1
            time.sleep(5)

        print("第",i,"家餐廳","\n\n")
        dic_emo[r_name].append("next")
    i+=1


['更：隔天再點一次|還是一堆炸焦|油吸飽的薯條|然後今天還多了一樣|鹽加太多超鹹活了大半輩子|吃了幾百次麥當勞|沒吃過像這樣子的|幾乎都丟掉沒吃 ' 0 no. 33
憤怒,失望,0.0,2.8, '乾淨明亮|店員很有禮貌與親切|內用吃早餐會讓人心情愉快 ' 1 no. 33
愉悅,信任,8.5,8.0,乾淨明亮 , 無
 '皮卡丘聯名|好可愛😍餐點跟以前一樣好吃|這裡的店員服務很親切哦👍 ' 2 no. 33
喜悅,信任,9.0,8.0,餐點好吃 , 無
 '可以認真一點嗎...已經打上去說不加起司了|打開還是看到起司|融化黏在裡面我也拿不掉|心情真的很糟 ' 3 no. 33
憤怒,擔心,0.0,3.5, '上回吃了一份麥脆雞|炸的很幹硬|我覺得是不是回鍋再炸過了|不然怎麼口感變差|早餐的無敵豬肉滿福堡就很不錯|薯餅也值得吃| ' 4 no. 33
 '五顆星給這輩子遇過最熱心的麥當勞店員|好像叫江世芳大姐|年紀稍長|主動教我們用app|非常謝謝您| ' 5 no. 33
感激,信任,9.0,8.0,熱心招待 , 無
 '優質薯條 現炸的\U0001faf6🏻資歷較深的女經理很有禮貌喔😁 … ' 6 no. 33
喜悅,信任,9.5,7.2,現炸薯條好吃 , 無
 '不小心翻倒食材|店員不但沒有生氣|還補翻倒的食物給我|這種服務態度怎麼能不推爆| ' 7 no. 33
 '假日習慣來這裡吃早餐|沒什麼人|很舒服|滿福堡加蛋套餐| ' 8 no. 33
舒適,喜悅,9.5,9.0,環境優美 , 無
 '前因不清楚但是看到主管在客人面前罵店員|和店員只能這樣擦地板...覺得好卑微|主管的脾氣身心要再修煉|生氣的同時也不要摔客人的餐點好嗎| ' 9 no. 33
憤怒,悲傷,3.8,2.5,店員被罵 , 主管態度差
 '一大早的開心的要吃個早餐卻遇到男店員的態度跟口氣都很不耐煩也很差是不是有起床氣|真的是連一顆星都不想給 ' 10 no. 33
憤怒,厭惡,0,2.5, '服務員態度很好|不論多忙都很有活力跟笑容|但忙碌的時候大薯份量會變這樣@@店內位置不多|所以我都外帶| ' 11 no. 33
喜悅,信任,8.5,7.2,服務佳 , 份量小
 '安格斯黑牛堡 肉都黑了還有焦味..薯條不知道怎麼回事怎麼跟外面路邊的一樣難吃很粉很乾..品質管理有待加強 ' 12 n

In [ ]:
i=1
for r_name in dic_cmt:
    print(i,len(restaurants[r_name]["emo"]),end="\t")
    if i%10==0:
        print("\n")
    i+=1

1 260	2 111	3 133	4 541	5 0	6 224	7 172	8 342	9 38	10 46	

11 124	12 543	13 426	14 210	15 56	16 476	17 86	18 264	19 48	20 106	

21 352	22 88	23 309	24 723	25 185	26 211	27 154	28 106	29 160	30 269	

31 340	32 138	33 97	34 94	35 156	36 235	37 134	38 286	39 606	40 577	

41 144	42 0	43 16	44 104	45 322	46 2	47 611	48 191	49 133	50 202	

51 601	52 512	53 92	54 84	55 138	56 76	57 545	58 128	59 10	60 306	

61 396	62 256	63 54	64 156	65 88	66 408	67 16	68 144	69 158	70 629	

71 142	72 576	73 341	74 132	75 80	76 176	77 79	78 257	79 130	80 123	

81 156	82 181	83 251	84 183	85 339	86 382	87 423	88 182	89 210	90 74	

91 118	92 283	93 120	94 260	95 174	96 118	97 0	98 0	

In [ ]:
import pandas as pd
import csv
fieldnames = ["restaurant_name", "href", "tag", "star","emo","score","good","bad"]
with open("emo_100.csv", mode='a',encoding="utf-8", newline="") as csv_file:
    writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
    for restaurant_name in restaurants:
        writer.writerow({
                    "restaurant_name": restaurant_name,
                    "href": restaurants[restaurant_name]["href"],
                    "tag": restaurants[restaurant_name]["tag"],
                    "star": restaurants[restaurant_name]["star"],

                    "emo": restaurants[restaurant_name]["emo"],
                    "score": restaurants[restaurant_name]["score"],
                    "good": restaurants[restaurant_name]["good"],
                    "bad": restaurants[restaurant_name]["bad"],
                })